In [ ]:
import pandas as pd
from transformers import pipeline
classification_task = pipeline("sentiment-analysis",model = "distilbert-base-uncased-finetuned-sst-2-english")
unmasker = pipeline('fill-mask', model='bert-base-uncased')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string 
result = string.punctuation 
from sentence_transformers import SentenceTransformer, util
import numpy as np
model = SentenceTransformer('stsb-roberta-large')
from tabulate import tabulate
import ipysheet
import csv
from timeit import default_timer as timer
import time
import homoglyphs as hg
from itertools import combinations
pyTorchData = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
bertAttackData = pd.read_csv('https://raw.githubusercontent.com/LinyangLee/BERT-Attack/master/data_defense/imdb_1k.tsv', delimiter='\t', header=None)
batch1=pyTorchData[0:10]
sentences = batch1[0]#here we can easily add the adversarial sentences

In [ ]:
def homoglyphAttack(word):
    listed = list(word)
    changes = []
    for j in range(len(listed)):
        possible = hg.Homoglyphs().get_combinations(listed[j])
        for k in possible:
            listed[j]=k
            changes.append("".join(listed))
        listed = list(word)
    return(changes)

In [ ]:
def charAttack(sentence,attack):
    if attack=="swap":
        everything = {}
        sentences = sentence
        sentences = sentences.replace(". ",".. ")
        sentences = sentences.replace("? ","?. ")
        sentences = sentences.replace("; ",";. ")
        sentences = sentences.replace("! ","!. ")
        sentenceList = sentences.split(". ")
        for z in range(len(sentenceList)):
            tempStorage = []
            originalSentence = sentenceList[z]
            originalLabel = classification_task(originalSentence)[0]["label"]#Get og sentence label
            originalScore = classification_task(originalSentence)[0]["score"]#get og sentence confidence
            originalEmbedding = model.encode(originalSentence, convert_to_tensor=True)#get vector of og sentence
            b = originalSentence#get another variable of the og sentence so when i change the og sentence punctuation i can revert it
            for k in originalSentence:#goes through all the characters of the og sentence
                if k in result:#if the character is a punctuation character then it is replaced with the same punctuation but surrounded by spaces
                    originalSentence = originalSentence.replace(k," "+k+" ")#the spaces are included so BERT knows that it is separate and is easier to split
            listed = originalSentence.split()#split the og sentence into a list. split by spaces 
            #initialize some lists
            different = []#one for when the label is different
            same = []#when the label is the same
            stopWords = []#used to house all the stop words
            a = False#used in case all words are stop words
            for i in range(len(listed)):#goes through all the words in the list
                if listed[i] not in stops:#if the word is not a stop word
                    listed = originalSentence.split()#make the list back into the og so this loop can run again
                    swappedWords = swapChar(listed[i])
                    for k in swappedWords:
                        changedWord = k
                        listed[i]=k
                        saved = ' '.join(listed)
                        changedSentence = saved#gets the changed sentence
                        changedLabel = classification_task(changedSentence)[0]["label"]#gets changed sentence label
                        changedScore = classification_task(changedSentence)[0]["score"]#gets changed sentence confidence level
                        changedEmbedding = model.encode(changedSentence, convert_to_tensor=True)#gets vector of the changed sentence
                        cosine_scores = util.pytorch_cos_sim(originalEmbedding, changedEmbedding)#gets the cosine score of the two vectors
                        #basically gets the similarity score of the two sentences
                        if changedLabel != originalLabel:#if the changed sentence label is different from the og label
                            #append it and all its information into a list of just when the label is different
                            different.append([changedSentence,changedLabel,changedScore,cosine_scores.item(),changedWord])

                        else:
                            #appends all the information when the label is the same
                            same.append([changedSentence,changedLabel,changedScore,cosine_scores.item(),changedWord])
                    saved = ''#restart the variables so the loop can run again
                    a=True#just meant to signify there is at least one example where there is information in the loops
                else:#if the word is a stop word
                    stopWords.append(listed[i])#append just that word into a list of just stop words
            if(a):#only run this portion if info in the lists exist

                if len(different)==0 and len(same)!=0:#if there are no instances of different labels
                    #initialize the info
                    minScore = same[0][2]
                    minSentence = same[0][0]
                    minWord = same[0][4]
                    minLabel = same[0][1]
                    minCosine = same[0][3]
                    simWord = same[0][4]
                    simScore = same[0][2]
                    simSentence = same[0][0]
                    simLabel = same[0][1]
                    simCosine = same[0][3]
                    for j in range(len(same)):
                        if same[j][3]>simCosine:#if there is a sentence with a higher similarity rating
                            #that its information
                            simScore = same[j][2]
                            simSentence = same[j][0]
                            simLabel = same[j][1]
                            simCosine = same[j][3]
                            simWord = same[j][4]
                            #then just print the info out
                        if minScore>same[j][2]:
                            minScore = same[j][2]
                            minSentence = same[j][0]
                            minLabel = same[j][1]
                            minCosine = same[j][3]
                            minWord = same[j][4]
                    
                    tempStorage.append(minSentence)
                    tempStorage.append(minWord)
                    tempStorage.append(minCosine)
                    
                elif len(different)!=0:#if there is atleast one sentence that has a different label
                    #initialize the info
                    maxScore = different[0][2]
                    maxSentence = different[0][0]
                    maxLabel = different[0][1]
                    maxCosine = different[0][3]
                    maxWord = different[0][4]
                    simSentence = different[0][0]
                    simScore = different[0][2]
                    simSentence = different[0][0]
                    simLabel = different[0][1]
                    simCosine = different[0][3]
                    simWord = different[0][4]
                    for j in range(len(different)):#goes through all the different label sentences
                        if different[j][3]>simCosine:#if its similarity score is higher
                            #take its information
                            simScore = different[j][2]
                            simSentence = different[j][0]
                            simLabel = different[j][1]
                            simCosine = different[j][3]
                            simWord = different[j][4]
                        if maxScore<different[j][2]:
                            maxScore = different[j][2]
                            maxSentence = different[j][0]
                            maxLabel = different[j][1]
                            maxCosine = different[j][3]   
                            maxWord = different[j][4]
                    #print all that info out
                    
                    tempStorage.append(maxSentence)
                    tempStorage.append(maxWord)
                    tempStorage.append(maxCosine)
            
            everything[z]=tempStorage  
    if attack=="homoglyph":
        everything = {}
        sentences = sentence
        sentences = sentences.replace(". ",".. ")
        sentences = sentences.replace("? ","?. ")
        sentences = sentences.replace("; ",";. ")
        sentences = sentences.replace("! ","!. ")
        sentenceList = sentences.split(". ")
        for z in range(len(sentenceList)):
            tempStorage = []
            originalSentence = sentenceList[z]
            originalLabel = classification_task(originalSentence)[0]["label"]#Get og sentence label
            originalScore = classification_task(originalSentence)[0]["score"]#get og sentence confidence
            originalEmbedding = model.encode(originalSentence, convert_to_tensor=True)#get vector of og sentence
            b = originalSentence#get another variable of the og sentence so when i change the og sentence punctuation i can revert it
            for k in originalSentence:#goes through all the characters of the og sentence
                if k in result:#if the character is a punctuation character then it is replaced with the same punctuation but surrounded by spaces
                    originalSentence = originalSentence.replace(k," "+k+" ")#the spaces are included so BERT knows that it is separate and is easier to split
            listed = originalSentence.split()#split the og sentence into a list. split by spaces 
            #initialize some lists
            different = []#one for when the label is different
            same = []#when the label is the same
            stopWords = []#used to house all the stop words
            a = False#used in case all words are stop words
            for i in range(len(listed)):#goes through all the words in the list
                if listed[i] not in stops:#if the word is not a stop word
                    listed = originalSentence.split()#make the list back into the og so this loop can run again
                    swappedWords = homoglyphAttack(listed[i])
                    for k in swappedWords:
                        changedWord = k
                        listed[i]=k
                        saved = ' '.join(listed)
                        changedSentence = saved#gets the changed sentence
                        changedLabel = classification_task(changedSentence)[0]["label"]#gets changed sentence label
                        changedScore = classification_task(changedSentence)[0]["score"]#gets changed sentence confidence level
                        changedEmbedding = model.encode(changedSentence, convert_to_tensor=True)#gets vector of the changed sentence
                        cosine_scores = util.pytorch_cos_sim(originalEmbedding, changedEmbedding)#gets the cosine score of the two vectors
                        #basically gets the similarity score of the two sentences
                        if changedLabel != originalLabel:#if the changed sentence label is different from the og label
                            #append it and all its information into a list of just when the label is different
                            different.append([changedSentence,changedLabel,changedScore,cosine_scores.item(),changedWord])

                        else:
                            #appends all the information when the label is the same
                            same.append([changedSentence,changedLabel,changedScore,cosine_scores.item(),changedWord])
                    saved = ''#restart the variables so the loop can run again
                    a=True#just meant to signify there is at least one example where there is information in the loops
                else:#if the word is a stop word
                    stopWords.append(listed[i])#append just that word into a list of just stop words
            if(a):#only run this portion if info in the lists exist

                if len(different)==0 and len(same)!=0:#if there are no instances of different labels
                    #initialize the info
                    minScore = same[0][2]
                    minSentence = same[0][0]
                    minWord = same[0][4]
                    minLabel = same[0][1]
                    minCosine = same[0][3]
                    simWord = same[0][4]
                    simScore = same[0][2]
                    simSentence = same[0][0]
                    simLabel = same[0][1]
                    simCosine = same[0][3]
                    for j in range(len(same)):
                        if same[j][3]>simCosine:#if there is a sentence with a higher similarity rating
                            #that its information
                            simScore = same[j][2]
                            simSentence = same[j][0]
                            simLabel = same[j][1]
                            simCosine = same[j][3]
                            simWord = same[j][4]
                            #then just print the info out
                        if minScore>same[j][2]:
                            minScore = same[j][2]
                            minSentence = same[j][0]
                            minLabel = same[j][1]
                            minCosine = same[j][3]
                            minWord = same[j][4]
                    
                    tempStorage.append(minSentence)
                    tempStorage.append(minWord)
                    tempStorage.append(minCosine)
                elif len(different)!=0:#if there is atleast one sentence that has a different label
                    #initialize the info
                    maxScore = different[0][2]
                    maxSentence = different[0][0]
                    maxLabel = different[0][1]
                    maxCosine = different[0][3]
                    maxWord = different[0][4]
                    simSentence = different[0][0]
                    simScore = different[0][2]
                    simSentence = different[0][0]
                    simLabel = different[0][1]
                    simCosine = different[0][3]
                    simWord = different[0][4]
                    for j in range(len(different)):#goes through all the different label sentences
                        if different[j][3]>simCosine:#if its similarity score is higher
                            #take its information
                            simScore = different[j][2]
                            simSentence = different[j][0]
                            simLabel = different[j][1]
                            simCosine = different[j][3]
                            simWord = different[j][4]
                        if maxScore<different[j][2]:
                            maxScore = different[j][2]
                            maxSentence = different[j][0]
                            maxLabel = different[j][1]
                            maxCosine = different[j][3]   
                            maxWord = different[j][4]
                    #print all that info out
                    
                    tempStorage.append(maxSentence)
                    tempStorage.append(maxWord)
                    tempStorage.append(maxCosine)
            everything[z]=tempStorage  
    if attack=="delete":
        everything = {}
        sentences = sentence
        sentences = sentences.replace(". ",".. ")
        sentences = sentences.replace("? ","?. ")
        sentences = sentences.replace("; ",";. ")
        sentences = sentences.replace("! ","!. ")
        sentenceList = sentences.split(". ")
        for z in range(len(sentenceList)):
            tempStorage = []
            originalSentence = sentenceList[z]
            originalLabel = classification_task(originalSentence)[0]["label"]#Get og sentence label
            originalScore = classification_task(originalSentence)[0]["score"]#get og sentence confidence
            originalEmbedding = model.encode(originalSentence, convert_to_tensor=True)#get vector of og sentence
            b = originalSentence#get another variable of the og sentence so when i change the og sentence punctuation i can revert it
            for k in originalSentence:#goes through all the characters of the og sentence
                if k in result:#if the character is a punctuation character then it is replaced with the same punctuation but surrounded by spaces
                    originalSentence = originalSentence.replace(k," "+k+" ")#the spaces are included so BERT knows that it is separate and is easier to split
            listed = originalSentence.split()#split the og sentence into a list. split by spaces 
            #initialize some lists
            different = []#one for when the label is different
            same = []#when the label is the same
            stopWords = []#used to house all the stop words
            a = False#used in case all words are stop words
            for i in range(len(listed)):#goes through all the words in the list
                if listed[i] not in stops:#if the word is not a stop word
                    listed = originalSentence.split()#make the list back into the og so this loop can run again
                    swappedWords = deleteAttack(listed[i])
                    for k in swappedWords:
                        changedWord = k
                        listed[i]=k
                        saved = ' '.join(listed)
                        changedSentence = saved#gets the changed sentence
                        changedLabel = classification_task(changedSentence)[0]["label"]#gets changed sentence label
                        changedScore = classification_task(changedSentence)[0]["score"]#gets changed sentence confidence level
                        changedEmbedding = model.encode(changedSentence, convert_to_tensor=True)#gets vector of the changed sentence
                        cosine_scores = util.pytorch_cos_sim(originalEmbedding, changedEmbedding)#gets the cosine score of the two vectors
                        #basically gets the similarity score of the two sentences
                        if changedLabel != originalLabel:#if the changed sentence label is different from the og label
                            #append it and all its information into a list of just when the label is different
                            different.append([changedSentence,changedLabel,changedScore,cosine_scores.item(),changedWord])

                        else:
                            #appends all the information when the label is the same
                            same.append([changedSentence,changedLabel,changedScore,cosine_scores.item(),changedWord])
                    saved = ''#restart the variables so the loop can run again
                    a=True#just meant to signify there is at least one example where there is information in the loops
                else:#if the word is a stop word
                    stopWords.append(listed[i])#append just that word into a list of just stop words
            if(a):#only run this portion if info in the lists exist

                if len(different)==0 and len(same)!=0:#if there are no instances of different labels
                    #initialize the info
                    minScore = same[0][2]
                    minSentence = same[0][0]
                    minWord = same[0][4]
                    minLabel = same[0][1]
                    minCosine = same[0][3]
                    simWord = same[0][4]
                    simScore = same[0][2]
                    simSentence = same[0][0]
                    simLabel = same[0][1]
                    simCosine = same[0][3]
                    for j in range(len(same)):
                        if same[j][3]>simCosine:#if there is a sentence with a higher similarity rating
                            #that its information
                            simScore = same[j][2]
                            simSentence = same[j][0]
                            simLabel = same[j][1]
                            simCosine = same[j][3]
                            simWord = same[j][4]
                            #then just print the info out
                        if minScore>same[j][2]:
                            minScore = same[j][2]
                            minSentence = same[j][0]
                            minLabel = same[j][1]
                            minCosine = same[j][3]
                            minWord = same[j][4]
                    
                    tempStorage.append(minSentence)
                    tempStorage.append(minWord)
                    tempStorage.append(minCosine)
                elif len(different)!=0:#if there is atleast one sentence that has a different label
                    #initialize the info
                    maxScore = different[0][2]
                    maxSentence = different[0][0]
                    maxLabel = different[0][1]
                    maxCosine = different[0][3]
                    maxWord = different[0][4]
                    simSentence = different[0][0]
                    simScore = different[0][2]
                    simSentence = different[0][0]
                    simLabel = different[0][1]
                    simCosine = different[0][3]
                    simWord = different[0][4]
                    for j in range(len(different)):#goes through all the different label sentences
                        if different[j][3]>simCosine:#if its similarity score is higher
                            #take its information
                            simScore = different[j][2]
                            simSentence = different[j][0]
                            simLabel = different[j][1]
                            simCosine = different[j][3]
                            simWord = different[j][4]
                        if maxScore<different[j][2]:
                            maxScore = different[j][2]
                            maxSentence = different[j][0]
                            maxLabel = different[j][1]
                            maxCosine = different[j][3]   
                            maxWord = different[j][4]
                    #print all that info out
                    
                    tempStorage.append(maxSentence)
                    tempStorage.append(maxWord)
                    tempStorage.append(maxCosine)
            everything[z]=tempStorage  
    if(not a):
        everything[0]=[sentence,"",0]
    return everything

In [ ]:
#run this if you want adversarial example
adversarialSentences = []
changedWords = []
cosineScores = []
runtimes = []
total = 0
counter = 0
for i in batch1[0]:
    start = timer()
    a=charAttack(i,"homoglyph")
    end = timer()
    runtimes.append(end-start)
    total+=(end-start)
    adversarialSentences.append(a[0][0])
    changedWords.append(a[0][1])
    cosineScores.append(a[0][2])
    counter+=1
    print(counter)
sentences = adversarialSentences
allLabels = {}
dictClassifications = {}
for i in range(len(sentences)):
    a = classification_task(sentences[i])
    dictClassifications[i]=a,sentences[i],batch1[1][i]
for i in dictClassifications:
    correct = False
    if dictClassifications[i][0][0]["label"] == "POSITIVE":
        predictedLabel = 1
    else:
        predictedLabel = 0
    givenLabel = dictClassifications[i][-1]
    if predictedLabel==givenLabel:
        correct = True
    allLabels[i]=sentences[i],dictClassifications[i][0],predictedLabel,givenLabel,correct,changedWords[i],cosineScores[i],runtimes[i]
    
totalWrong = 0
wrongSentences = []
for i in allLabels:
    correct = allLabels[i][-2]
    if not correct:
        wrongLabel = allLabels[i][1][0]["label"]
        wrongConfidence = allLabels[i][1][0]["score"]
        wrongSentences.append((sentences[i],wrongLabel,wrongConfidence,i))
        
        totalWrong+=1
percentageRight = (len(allLabels)-totalWrong)/len(allLabels)    

In [ ]:
col_names = ["Sentence", "Actual Label","Predicted Label","Confidence","Accuracy","Changed Word","Cosine Similarity Score","Runtimes(s)"]
data = []
for i in range(len(allLabels)):
    sentence = allLabels[i][0]
    changedWord = allLabels[i][5]
    confidence = allLabels[i][1][0]['score']
    similarity = allLabels[i][6]
    runtime = allLabels[i][7]
    if allLabels[i][2]==1:
        predicted = 'Positive'
    elif allLabels[i][2]==0:
        predicted = 'Negative'
    if allLabels[i][3]==1:
        actual = 'Positive'
    elif allLabels[i][3]==0:
        actual = 'Negative'
    if allLabels[i][4]:
        correct = "Correct"
    else:
        correct = "False"
    data.append([sentence,actual,predicted,confidence,correct,changedWord,similarity,runtime])
    


In [ ]:
truePositives = []
trueNegatives = []
falsePositives = []
falseNegatives = []
for i in range(len(data)):
    if data[i][4]=="False":
        if data[i][2]=='Positive':
            falsePositives.append(i)
        else:
            falseNegatives.append(i)
    else:
        if data[i][2]=='Positive':
            truePositives.append(i)
        else:
            trueNegatives.append(i)

In [ ]:
numTruePositives = len(truePositives)
numTrueNegatives = len(trueNegatives)
numFalsePositives = len(falsePositives)
numFalseNegatives = len(falseNegatives)
accuracy = (numTruePositives+numTrueNegatives)/(numTruePositives+numTrueNegatives+numFalsePositives+numFalseNegatives)
precision = numTruePositives/(numTruePositives+numFalsePositives)
recall = numTruePositives/(numTruePositives+numFalseNegatives)
specificity =numTrueNegatives/(numTrueNegatives+numFalsePositives)
f1 = 2*(precision*recall)/(precision+recall)

In [ ]:
filename = "Adversarial_Char_Homoglyph_PyTorch_VPN.csv"
col_names = ["Sentence", "Actual Label","Predicted Label","Confidence","Accuracy","Changed Word","Cosine Similarity Score","Runtimes(s)"]
row1 = [f"Accuracy:{accuracy:.3f}",f"Precision:{precision:.3f}",f"Recall:{recall:.3f}",f"Specificity:{specificity:.3f}",f"F1-Score:{f1:.3f}",f"Total Runtime(s):{total:.3f}"]
with open(filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
     
    # writing the fields
    csvwriter.writerow(row1)
    csvwriter.writerow(col_names)
     
    # writing the data rows
    csvwriter.writerows(data)